# Notebook to demonstrate Zero shot and Few shot Learning

In [1]:
import pandas as pd 
from langchain_groq.chat_models import ChatGroq
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report
from apikey import api_key

In [2]:
# Groq API and Models 
Groq_Token = api_key  # Do not share this key with anyone

groq_models = {"llama3-70b": "llama3-70b-8192", "mixtral": "mixtral-8x7b-32768", "gemma-7b": "gemma-7b-it","llama3.1-70b":"llama-3.1-70b-versatile","llama3-8b":"llama3-8b-8192","llama3.1-8b":"llama-3.1-8b-instant","gemma-9b":"gemma2-9b-it"}

**NOTE : DO NOT SHARE THE API KEY WITH ANYONE. DO NOT COMMIT THE API KEY TO GITHUB.**

Always do a sanity check before committing the code to github. If the key is found in the code, you will be penalized with a 0.5 marks deduction.

# Zero Shot 

In [12]:
# Statement 
sentence = "The product quality is amazing but the delivery was delayed. However I am happy with the customer service."

# System Prompts 
query = f"""
* You are a sentiment analysis model. 
* Your task is to analyze the sentiment expressed in the given text and classify it as 'positive', 'negative', or 'neutral'. 
* Provide the sentiment label and, if necessary, a brief explanation of your reasoning.

Sentence: {sentence}
""" 

# To use Groq LLMs 
model_name = "llama3-70b" # We can choose any model from the groq_models dictionary
llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
answer = llm.invoke(query)

print(answer.content)

Sentiment label: Neutral

Explanation: The sentence expresses mixed sentiments. The words "amazing" and "happy" convey a positive sentiment, indicating satisfaction with the product quality and customer service, respectively. However, the phrase "delivery was delayed" expresses a negative sentiment, indicating dissatisfaction with the delivery experience. Since both positive and negative sentiments are present, the overall sentiment is neutral.


# Few Shot

In [13]:
# Statement 
sentence = "The product quality is amazing but the delivery was delayed. However I am happy with the customer service."

# System Prompts 
query = f"""
* You are a sentiment analysis model. 
* Your task is to analyze the sentiment expressed in the given text and classify it as 'positive', 'negative', or 'neutral'. 
* Provide the sentiment label and, if necessary, a brief explanation of your reasoning.

Here are few examples:
1. Sentence: 'The customer service was excellent, and I received my order quickly.'
Sentiment: Positive

2. Sentence: 'The food was bland and the service was slow.'
Sentiment: Negative

3. Sentence: 'The product is okay, but it's not worth the price.'
Sentiment: Neutral

Sentence: {sentence}
""" 

# To use Groq LLMs 
model_name = "llama3-70b" # We can choose any model from the groq_models dictionary
llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
answer = llm.invoke(query)

print(answer.content)

Sentiment: Positive

Explanation: Although the sentence mentions a negative aspect ("the delivery was delayed"), the positive sentiments expressed in the sentence ("The product quality is amazing" and "I am happy with the customer service") outweigh the negative one, resulting in an overall positive sentiment.


3

In [7]:
#=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
#
#                                   ES335- Machine Learning- Assignment 1
#
# This file is used to create the dataset for the mini-project. The dataset is created by reading the data from
# the Combined folder. The data is then split into training, testing, and validation sets. This split is supposed
# to be used for all the modeling purposes.
#
#=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

# Library imports
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import os

# Constants
time = 10
offset = 100
folders = ["LAYING","SITTING","STANDING","WALKING","WALKING_DOWNSTAIRS","WALKING_UPSTAIRS"]
classes = {"WALKING":1,"WALKING_UPSTAIRS":2,"WALKING_DOWNSTAIRS":3,"SITTING":4,"STANDING":5,"LAYING":6}

combined_dir = os.path.join("Combined")

#=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-==-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
                                                # Train Dataset
#=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-==-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

X_train=[]
y_train=[]
dataset_dir = os.path.join(combined_dir,"Train")

for folder in folders:
    files = os.listdir(os.path.join(dataset_dir,folder))

    for file in files:

        df = pd.read_csv(os.path.join(dataset_dir,folder,file),sep=",",header=0)
        df = df[offset:offset+time*50]
        X_train.append(df.values)
        y_train.append(classes[folder])

X_train = np.array(X_train)
y_train = np.array(y_train)


#=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-==-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
                                                # Test Dataset
#=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-==-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

X_test=[]
y_test=[]
dataset_dir = os.path.join(combined_dir,"Test")

for folder in folders:
    files = os.listdir(os.path.join(dataset_dir,folder))
    for file in files:

        df = pd.read_csv(os.path.join(dataset_dir,folder,file),sep=",",header=0)
        df = df[offset:offset+time*50]
        X_test.append(df.values)
        y_test.append(classes[folder])

X_test = np.array(X_test)
y_test = np.array(y_test)

#=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-==-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
                                                # Final Dataset
#=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-==-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

# USE THE BELOW GIVEN DATA FOR TRAINING and TESTING purposes

# concatenate the training and testing data
X = np.concatenate((X_train,X_test))
y = np.concatenate((y_train,y_test))

# split the data into training and testing sets. Change the seed value to obtain different random splits.
seed = 4
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=seed,stratify=y)

print("Training data shape: ",X_train.shape)
print("Testing data shape: ",X_test.shape)

#=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-==-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=




Training data shape:  (126, 500, 3)
Testing data shape:  (54, 500, 3)


In [8]:
data = X_train[0]
y_train[0]

np.int64(2)

In [9]:
import tsfel
from sklearn import preprocessing
cfg = tsfel.get_features_by_domain() # retrieves all features
X_train_featurised = tsfel.time_series_features_extractor(cfg, X_train, fs=50)
X_test_featurised = tsfel.time_series_features_extractor(cfg, X_test, fs=50)

In [17]:
from sklearn.decomposition import PCA
pca = PCA(n_components=10)
X_train_reduced_tsefl = pca.fit_transform(X_train_featurised)
print(X_train_featurised.shape)
print(X_train_reduced_tsefl.shape)

(126, 495)
(126, 10)


3.1.1

In [18]:
# System Prompts 
query = """ 
You are tasked with classifying human activities based on featurized accelerometer data. The activities include:
- WALKING
- WALKING_UPSTAIRS
- WALKING_DOWNSTAIRS
- SITTING
- STANDING
- LAYING

Now, given the following 495-feature vector representing an activity window, predict the most likely activity label:
Feature Vector: {featues}


do not give code.
do not give extra information.
return predicted activity only.
"""

# To use Groq LLMs 
model_name = "llama3-70b" # We can choose any model from the groq_models dictionary
llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
predicted_activities = []
for i in range(30):  # Predict for 30 examples
    example_vector = X_train_reduced_tsefl[i+3].tolist()  # Using different test examples
    prompt = query.format(featues=example_vector)
    predicted_activity = llm.invoke(prompt)
    predicted_activities.append(classes[predicted_activity.content])
    print(f"Predicted Activity for example {i+1}: {classes[predicted_activity.content]}")
print(predicted_activities)

Predicted Activity for example 1: 1
Predicted Activity for example 2: 1
Predicted Activity for example 3: 1
Predicted Activity for example 4: 1
Predicted Activity for example 5: 1
Predicted Activity for example 6: 1
Predicted Activity for example 7: 1
Predicted Activity for example 8: 1
Predicted Activity for example 9: 1
Predicted Activity for example 10: 1
Predicted Activity for example 11: 1
Predicted Activity for example 12: 1
Predicted Activity for example 13: 1
Predicted Activity for example 14: 1
Predicted Activity for example 15: 1
Predicted Activity for example 16: 1
Predicted Activity for example 17: 1
Predicted Activity for example 18: 1
Predicted Activity for example 19: 1
Predicted Activity for example 20: 1
Predicted Activity for example 21: 1
Predicted Activity for example 22: 1
Predicted Activity for example 23: 1
Predicted Activity for example 24: 1
Predicted Activity for example 25: 1
Predicted Activity for example 26: 1
Predicted Activity for example 27: 1
Predicted 

In [19]:
y_pred = np.array(predicted_activities)

y_pred.shape
accuracy = accuracy_score(y_test[3:33], y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.1667


3.1.2

In [14]:
few_shot_prompt = """ 
You are tasked with classifying human activities based on featurized accelerometer data. The activities include:
- WALKING
- WALKING_UPSTAIRS
- WALKING_DOWNSTAIRS
- SITTING
- STANDING
- LAYING

Here are a few labeled examples of the feature vectors and their corresponding activities:

Example 1:
Feature Vector: {example_1_vector}
Activity: {example_1_label}

Example 2:
Feature Vector: {example_2_vector}
Activity: {example_2_label}

Example 3:
Feature Vector: {example_3_vector}
Activity: {example_3_label}

Now, given the following 495-feature vector representing an activity window, predict the most likely activity label:
Feature Vector: {feature_vector}


do not give code.
do not give extra information.
return predicted activity only.
"""

# Few-shot learning function
def few_shot_classification(feature_vector, labeled_examples):
    # Prepare the prompt using a few labeled examples
    prompt = few_shot_prompt.format(
        example_1_vector=labeled_examples[0]["feature_vector"],
        example_1_label=labeled_examples[0]["label"],
        example_2_vector=labeled_examples[1]["feature_vector"],
        example_2_label=labeled_examples[1]["label"],
        example_3_vector=labeled_examples[2]["feature_vector"],
        example_3_label=labeled_examples[2]["label"],
        feature_vector=feature_vector
    )
    
    # Invoke the Groq LLM
    predicted_label = llm.invoke(prompt)
    return predicted_label

# Example labeled data for few-shot learning
labeled_examples = [
    {"feature_vector": X_train_reduced_tsefl[0].tolist(), "label": "WALKING_UPSTAIRS"},
    {"feature_vector": X_train_reduced_tsefl[1].tolist(), "label": "LAYING"},
    {"feature_vector": X_train_reduced_tsefl[2].tolist(), "label": "STANDING"},
    {"feature_vector": X_train_reduced_tsefl[6].tolist(), "label": "WALKING"},
    {"feature_vector": X_train_reduced_tsefl[16].tolist(), "label": "WALKING_DOWNSTAIRS"},
    {"feature_vector": X_train_reduced_tsefl[9].tolist(), "label": "SITTING"},
]
predicted_activities = []
for i in range(30):  # Predict for 30 examples
    example_vector = X_train_reduced_tsefl[i + 3].tolist()  # Using different test examples
    predicted_activity = few_shot_classification(example_vector, labeled_examples)
    predicted_activities.append(classes[predicted_activity.content])
    print(f"Predicted Activity for example {i+1}: {classes[predicted_activity.content]}")

Predicted Activity for example 1: 2
Predicted Activity for example 2: 3
Predicted Activity for example 3: 3
Predicted Activity for example 4: 2
Predicted Activity for example 5: 3
Predicted Activity for example 6: 2
Predicted Activity for example 7: 2
Predicted Activity for example 8: 3
Predicted Activity for example 9: 1
Predicted Activity for example 10: 2
Predicted Activity for example 11: 2
Predicted Activity for example 12: 6
Predicted Activity for example 13: 3
Predicted Activity for example 14: 3
Predicted Activity for example 15: 2
Predicted Activity for example 16: 3
Predicted Activity for example 17: 5
Predicted Activity for example 18: 3
Predicted Activity for example 19: 3
Predicted Activity for example 20: 2
Predicted Activity for example 21: 6
Predicted Activity for example 22: 2
Predicted Activity for example 23: 2
Predicted Activity for example 24: 3
Predicted Activity for example 25: 6
Predicted Activity for example 26: 5
Predicted Activity for example 27: 5
Predicted 

In [15]:
y_pred = np.array(predicted_activities)

y_pred.shape
accuracy = accuracy_score(y_test[3:33], y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.2000


decision tree has more accuracy than fewshot 